In [1]:
!pip install torch==2.6.0 torchvision==0.21.0
!pip install transformers==4.48.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.0 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 9.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 10.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 6.7 MB/s eta 0:00:000:00:0100:

In [2]:
!pip install accelerate==1.3.0
!pip install soundfile==0.13.1
!pip install pillow==11.1.0
!pip install scipy==1.15.2
!pip install backoff==2.2.1
!pip install peft==0.13.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


In [3]:
!pip install backoff==2.2.1

In [4]:
import os
import json
import requests
import tqdm
import pandas as pd
import torch
from PIL import Image
import soundfile
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

2025-04-27 16:57:13.604903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745773033.798985      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745773033.853665      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig


model_path = "microsoft/Phi-4-multimodal-instruct"

kwargs = {}
kwargs['torch_dtype'] = torch.bfloat16

processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
print(processor.tokenizer)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype='auto',
    _attn_implementation='eager',
    low_cpu_mem_usage=True,
    device_map = "auto"
).cuda()
print("model.config._attn_implementation:", model.config._attn_implementation)

generation_config = GenerationConfig.from_pretrained(model_path, 'generation_config.json')

user_prompt = '<|user|>'
assistant_prompt = '<|assistant|>'
prompt_suffix = '<|end|>'


GPT2TokenizerFast(name_or_path='microsoft/Phi-4-multimodal-instruct', vocab_size=200019, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	199999: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200010: AddedToken("<|endoftext10|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200011: AddedToken("<|endoftext11|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200018: AddedToken("<|endofprompt|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200019: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200020: AddedToken("<|end|>", rstrip=T

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model.config._attn_implementation: eager


In [6]:
!pip install backoff

In [14]:
prompt = """
You are given an image of a handwritten medical prescription. 
Extract the following structured information(if available):
- Doctor Name
- Clinic Address
- Patient Name
- Patient Age
- Address (if available)
- Date
- Medicine Name(s)
- Dosage
- Frequency
- Duration
Format output strictly as JSON.
    """

In [15]:
def extract_info(image_path , prompt = prompt):
    prompt = f'{user_prompt}<|image_1|>{prompt}{prompt_suffix}{assistant_prompt}'
    image = Image.open(image_path)
    inputs = processor(text=prompt, images=image, return_tensors='pt').to('cuda:0')

    output = generate(inputs)
    
    return output

def generate(inputs):
    generate_ids = model.generate(
        **inputs,
        max_new_tokens=1000,
        generation_config=generation_config,
    )
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1] :]
    response = processor.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    return response

In [16]:
image_folder = '/kaggle/input/illegible-medical-prescription-images-dataset/data'
output_folder = '/kaggle/working/outputs'
os.makedirs(output_folder, exist_ok=True)

In [17]:
results = []

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


In [23]:
extract_info(os.path.join(image_folder, image_files[10]))

'{"Doctor Name": "Dr. S.", "Clinic Address": "Pediatric Clinic", "Patient Name": "Sonia", "Patient Age": "7/19", "Address": "98 cm, 98, 102", "Date": "7/19", "Medicine Name(s)": ["Epopressor", "Syp crecin", "Syp rosefetal"], "Dosage": ["4ml", "4ml", "4ml"], "Frequency": ["twice (00)", "twice (00)", "twice (00)"], "Duration": "4ml"}'

In [22]:
image_files[0]

'45.jpg'

In [ ]:
for img_file in tqdm.tqdm(image_files):
    img_path = os.path.join(image_folder, img_file)
    try:
        output = extract_info(img_path)
        results.append({
            'image': img_file,
            'extracted_info': output
        })
    except Exception as e:
        print(f"Error processing {img_file}: {e}")

  0%|          | 0/129 [00:00<?, ?it/s]

In [30]:
with open(os.path.join(output_folder, 'extracted_data.json'), 'w') as f:
    json.dump(results, f, indent=4)

# As CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(output_folder, 'extracted_data.csv'), index=False)

In [31]:
df

,image,extracted_info
0,45.jpg,"{""Doctor Name"": ""Dr. Shikhar Joon"", ""Clinic Ad..."
1,56.jpg,"{""Clinic Address"": ""500-600 W 4TH STREET, ODES..."
2,45.jpg,"{""Doctor Name"": ""Dr. Shikhar Joon"", ""Clinic Ad..."
3,56.jpg,"{""Clinic Address"": ""500-600 W 4TH STREET, ODES..."
4,89.jpg,"{""Clinic Address"": ""500 - 600 W. 4TH STREET, O..."
5,20.jpg,"{""Doctor Name"": ""MIDTOWN PARK PHARMACY"", ""Clin..."
6,58.jpg,"{""Doctor Name"": ""R"", ""Clinic Address"": ""3ml 6D..."
